In [ ]:
import numpy as np
import pandas as pd
import os
import subprocess
import shutil
from pathlib import Path

def setup_freesurfer():
    """Initialize FreeSurfer environment"""
    freesurfer_home = '/home/localadmin/freesurfer'
    subjects_dir = "/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia"
    
    os.environ['FREESURFER_HOME'] = freesurfer_home
    os.environ['SUBJECTS_DIR'] = subjects_dir
    
    setup_cmd = f"bash -c 'source {freesurfer_home}/SetUpFreeSurfer.sh; env'"
    try:
        process = subprocess.Popen(setup_cmd, stdout=subprocess.PIPE, shell=True)
        output, _ = process.communicate()
        
        for line in output.decode().split('\n'):
            if '=' in line:
                key, value = line.split('=', 1)
                os.environ[key] = value
        
        print(f"FreeSurfer environment initialized successfully")
        print(f"FREESURFER_HOME: {os.environ.get('FREESURFER_HOME')}")
        print(f"SUBJECTS_DIR: {os.environ.get('SUBJECTS_DIR')}")  # Add this line
    except Exception as e:
        print(f"Error setting up FreeSurfer: {e}")
        raise
    
def verify_files(y_file, X_file, contrast_mat):
    """Verify that input files exist and are readable"""
    files_to_check = [
        (y_file, "Y file"),
        (X_file, "Design matrix"),
        (contrast_mat, "Contrast matrix")
    ]
    
    for file_path, file_type in files_to_check:
        if not os.path.exists(file_path):
            print(f"Error: {file_type} not found at {file_path}")
            return False
        
        if not os.access(file_path, os.R_OK):
            print(f"Error: {file_type} is not readable at {file_path}")
            return False
        
        # Print file size
        size = os.path.getsize(file_path)
        print(f"{file_type} size: {size} bytes")
    
    return True

def run_glmfit(y_file, X_file, output_dir, contrast_mat):
    """
    Run mri_glmfit with specified parameters
    Parameters:
    -----------
    y_file : str
    Path to input .mgh file
    X_file : numpy.ndarray
    Design matrix
    output_dir : str
    Directory for output files
    contrast_mat : str, optional
    Path to contrast matrix file
    surf : tuple, optional
    (subject, hemisphere) for surface analysis
    """

    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Extract hemisphere (lh or rh) from y_file name
    y_name = Path(y_file).stem  # e.g., 'group_lh_ad'
    hemi = 'lh' if 'lh' in y_name else 'rh'
    
    # Build command
    cmd = [
    'mri_glmfit',
    '--y', y_file,
    '--X', X_file,
    '--glmdir', output_dir,
    '--C', contrast_mat,
    '--surf', 'fsaverage', hemi,
    '--eres-save'
    ]

    # Run command
    try:
        print("Running mri_glmfit with command:", ' '.join(cmd))
        
        # Add shell=True to see more detailed error messages
        result = subprocess.run(
            ' '.join(cmd),
            shell=True,
            check=True,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
            env=os.environ  # Explicitly pass environment variables
        )
        print("Success! Output written to:", output_dir)
        print("Output:", result.stdout)  # Print stdout
        return True, result.stdout
    except subprocess.CalledProcessError as e:
        print("Error running mri_glmfit:")
        print("Return code:", e.returncode)
        print("Error output:", e.stderr)
        print("Standard output:", e.stdout)  # Add this line
        return False, e.stderr
    except Exception as e:
        print("Unexpected error:", str(e))
        print("Type of error:", type(e))  # Add this line
        return False, str(e)
    
def run_multiple_glms(y_files, X_file, contrast_files, base_output_dir):
    """Run multiple GLMs with verification"""
    
    # Create base output directory
    os.makedirs(base_output_dir, exist_ok=True)
    
    for y_file in y_files:
        y_name = Path(y_file).stem
        
        for contrast_file in contrast_files:
            contrast_name = Path(contrast_file).stem.split('_')[-1]
            glm_dir = os.path.join(base_output_dir, f"{y_name}_{contrast_name}.glmdir")

            print(f"\nProcessing GLM for {y_name} with contrast {contrast_name}")
            
            # Verify files before running
            if not verify_files(y_file, X_file, contrast_file):
                print(f"Skipping GLM for {y_name} due to file verification failure")
                continue
                
            # Try to read and print content of design and contrast matrices
            try:
                with open(X_file, 'r') as f:
                    print("\nDesign matrix preview:")
                    print(f.read().strip()[:200] + "...")  # Print first 200 chars
                
                with open(contrast_file, 'r') as f:
                    print("\nContrast matrix preview:")
                    print(f.read().strip()[:200] + "...")  # Print first 200 chars
            except Exception as e:
                print(f"Error reading matrix files: {e}")
            
            success, output = run_glmfit(
                y_file=y_file,
                X_file=X_file,
                output_dir=glm_dir,
                contrast_mat=contrast_file
            )
            
# Example usage
if __name__ == "__main__":
    
    # Initialize FreeSurfer
    setup_freesurfer()
    
    # Example data
    X_file = "/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/glm/group_design.mat"
    
    y_files = [
        "/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/new_group_analysis/group_lh_ad_fwhm6_demeaned.mgh",
        "/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/new_group_analysis/group_lh_md_fwhm6_demeaned.mgh",
        "/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/new_group_analysis/group_lh_rd_fwhm6_demeaned.mgh",
        "/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/new_group_analysis/group_lh_ak_fwhm6_demeaned.mgh",
        "/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/new_group_analysis/group_lh_mk_fwhm6_demeaned.mgh",
        "/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/new_group_analysis/group_lh_rk_fwhm6_demeaned.mgh",
        "/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/new_group_analysis/group_lh_fa_fwhm6_demeaned.mgh",
        "/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/new_group_analysis/group_rh_ad_fwhm6_demeaned.mgh",
        "/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/new_group_analysis/group_rh_md_fwhm6_demeaned.mgh",
        "/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/new_group_analysis/group_rh_rd_fwhm6_demeaned.mgh",
        "/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/new_group_analysis/group_rh_ak_fwhm6_demeaned.mgh",
        "/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/new_group_analysis/group_rh_mk_fwhm6_demeaned.mgh",
        "/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/new_group_analysis/group_rh_rk_fwhm6_demeaned.mgh",
        "/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/new_group_analysis/group_rh_fa_fwhm6_demeaned.mgh",
        "/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/new_group_analysis/group_lh_ad_fwhm10_demeaned.mgh",
        "/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/new_group_analysis/group_lh_md_fwhm10_demeaned.mgh",
        "/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/new_group_analysis/group_lh_rd_fwhm10_demeaned.mgh",
        "/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/new_group_analysis/group_lh_ak_fwhm10_demeaned.mgh",
        "/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/new_group_analysis/group_lh_mk_fwhm10_demeaned.mgh",
        "/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/new_group_analysis/group_lh_rk_fwhm10_demeaned.mgh",
        "/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/new_group_analysis/group_lh_fa_fwhm10_demeaned.mgh",
        "/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/new_group_analysis/group_rh_ad_fwhm10_demeaned.mgh",
        "/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/new_group_analysis/group_rh_md_fwhm10_demeaned.mgh",
        "/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/new_group_analysis/group_rh_rd_fwhm10_demeaned.mgh",
        "/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/new_group_analysis/group_rh_ak_fwhm10_demeaned.mgh",
        "/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/new_group_analysis/group_rh_mk_fwhm10_demeaned.mgh",
        "/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/new_group_analysis/group_rh_rk_fwhm10_demeaned.mgh",
        "/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/new_group_analysis/group_rh_fa_fwhm10_demeaned.mgh",
    ]
    
    contrast_files = [
        "/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/glm/group_contrast_HCEP.mat",
        "/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/glm/group_contrast_EPHC.mat",
    ]
    
    base_output_dir = "/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/new_group_results"

    run_multiple_glms(y_files, X_file, contrast_files, base_output_dir)